<a href="https://colab.research.google.com/github/singh-12345/Plagiarism_Checker_AI/blob/main/Plagiarism_Checker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import nltk
nltk.download('popular')
from nltk.corpus import stopwords
import string
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [3]:
df = pd.read_csv("train_snli.txt", sep="\t", header=None, names=["input_text", "plagiarized_text", "label"])
df.head()

,input_text,plagiarized_text,label
0,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",0
1,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",1
2,Children smiling and waving at camera,There are children present,1
3,Children smiling and waving at camera,The kids are frowning,0
4,A boy is jumping on skateboard in the middle o...,The boy skates down the sidewalk.,0


In [4]:
print(df['label'].value_counts())
# 0 -> Not Plagiarised
# 1 -> Plagiarised

label
1    1050
0    1049
Name: count, dtype: int64


In [5]:
df.shape

(2099, 3)

In [6]:
df = df.sample(frac=0.5, random_state=42)  # Keep only 50% of the data
df = df.head(1000)  # Keep only the first 1000 rows
df = df.iloc[:500]  # Keep only the first 500 rows
df.shape

(500, 3)

In [7]:
import string
import nltk
from nltk.corpus import stopwords
import pandas as pd
nltk.download("stopwords")

def process_text(text):
    if not isinstance(text, str):  # Check if input is string
        return ""
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = text.lower()
    stop_words = set(stopwords.words('english'))
    text = " ".join((word for word in text.split() if word not in stop_words))
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
df['input_text']=df['input_text'].apply(process_text)
df['plagiarized_text']=df['plagiarized_text'].apply(process_text)

In [9]:
df

,input_text,plagiarized_text,label
1005,man riding dirt bike,human riding vehicle,1
1078,man gray vehicle feeding sheep,man outside,1
67,woman walking across street eating banana man ...,woman eating banana,1
867,children playing game field,children playing game,1
650,man bright green shirt shows woman bright pink...,man woman,1
...,...,...,...
532,brown dog treads water,brown dog treads water soaked water,1
607,young woman running two guys back try catch,group people running,1
394,exhausted looking firemen walking,firemen walking,1
930,family van fence,family pickup truck,0


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Fill NaN values with an empty string
df["input_text"] = df["input_text"].fillna("").astype(str)
df["plagiarized_text"] = df["plagiarized_text"].fillna("").astype(str)

# Combine both columns into one
df["combined_text"] = df["input_text"] + " " + df["plagiarized_text"]

# Initialize and transform text data
tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df["combined_text"])

# Print shape of the resulting matrix
print("TF-IDF Matrix Shape:", X.shape)

TF-IDF Matrix Shape: (500, 1316)


In [11]:
df

,input_text,plagiarized_text,label,combined_text
1005,man riding dirt bike,human riding vehicle,1,man riding dirt bike human riding vehicle
1078,man gray vehicle feeding sheep,man outside,1,man gray vehicle feeding sheep man outside
67,woman walking across street eating banana man ...,woman eating banana,1,woman walking across street eating banana man ...
867,children playing game field,children playing game,1,children playing game field children playing game
650,man bright green shirt shows woman bright pink...,man woman,1,man bright green shirt shows woman bright pink...
...,...,...,...,...
532,brown dog treads water,brown dog treads water soaked water,1,brown dog treads water brown dog treads water ...
607,young woman running two guys back try catch,group people running,1,young woman running two guys back try catch gr...
394,exhausted looking firemen walking,firemen walking,1,exhausted looking firemen walking firemen walking
930,family van fence,family pickup truck,0,family van fence family pickup truck


In [12]:
y = df['label']

In [13]:
# train test split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [14]:
# logistic regression
model = LogisticRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

print("accuracy" , accuracy_score(y_test,y_pred))
print("Classification ", classification_report(y_test,y_pred))
print("confusion ", confusion_matrix(y_test,y_pred))

accuracy 0.47
Classification                precision    recall  f1-score   support

           0       0.53      0.38      0.44        55
           1       0.43      0.58      0.50        45

    accuracy                           0.47       100
   macro avg       0.48      0.48      0.47       100
weighted avg       0.48      0.47      0.47       100

confusion  [[21 34]
 [19 26]]


In [15]:
# Random forest

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize and train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Display results
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.44

Classification Report:
               precision    recall  f1-score   support

           0       0.49      0.35      0.40        55
           1       0.41      0.56      0.47        45

    accuracy                           0.44       100
   macro avg       0.45      0.45      0.44       100
weighted avg       0.45      0.44      0.43       100


Confusion Matrix:
 [[19 36]
 [20 25]]


In [16]:
# naive bays

from sklearn.naive_bayes import MultinomialNB


model = MultinomialNB()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

print("accuracy" , accuracy_score(y_test,y_pred))
print("Classification ", classification_report(y_test,y_pred))
print("confusion ", confusion_matrix(y_test,y_pred))

accuracy 0.44
Classification                precision    recall  f1-score   support

           0       0.48      0.27      0.35        55
           1       0.42      0.64      0.51        45

    accuracy                           0.44       100
   macro avg       0.45      0.46      0.43       100
weighted avg       0.46      0.44      0.42       100

confusion  [[15 40]
 [16 29]]


In [17]:
# SVM

# Import libraries
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from scipy.sparse import csr_matrix

# Convert to dense matrix if needed
if isinstance(X_train, csr_matrix):
    X_train = X_train.toarray()
    X_test = X_test.toarray()

# Scale the data (with_mean=False for sparse matrices)
scaler = StandardScaler(with_mean=False)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train SVM model
model = SVC(kernel='linear', random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Print results
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.48

Classification Report:
               precision    recall  f1-score   support

           0       0.54      0.36      0.43        55
           1       0.44      0.62      0.52        45

    accuracy                           0.48       100
   macro avg       0.49      0.49      0.48       100
weighted avg       0.50      0.48      0.47       100


Confusion Matrix:
 [[20 35]
 [17 28]]


In [18]:
import pickle
pickle.dump(model,open('model.pkl','wb'))
pickle.dump(tfidf_vectorizer,open('tfidf_vectorizer.pkl','wb'))

In [19]:
import pickle
model = pickle.load(open('model.pkl','rb'))
tfidf_vectorizer = pickle.load(open('tfidf_vectorizer.pkl','rb'))

In [20]:
# detection system

import numpy as np

def detect(input_txt):
    vectorized_text = tfidf_vectorizer.transform([input_txt])
    vectorized_text = vectorized_text.toarray()  # Convert sparse matrix to dense array

    result = model.predict(vectorized_text)

    print(f"Predicted Value: {result[0]}")  # Debugging output

    return "Plagiarism Detected" if result[0] == 1 else "No Plagiarism"

# Custom input
input_txt = input("Enter a sentence to check for plagiarism: ")
print(detect(input_txt))


Enter a sentence to check for plagiarism: this is pen
Predicted Value: 1
Plagiarism Detected
